In [1]:

import sys
import pylab as pl
# import scipy.sparse
# import scipy.sparse.linalg
import numpy as np
# from scipy import linalg
import random as rnd
import os
import pandas as pd
import time as tm
import numpy.linalg as nplin
import scipy.sparse
import scipy.sparse.linalg
# import scipy.linalg as sclin
import random 
import os

In [2]:
#!/usr/bin/env python
# coding: utf-8



Thetta= [[70,50]]
thetp = list([Thetta[i][0] for i in range( len(Thetta))])
thetd = list([Thetta[i][1] for i in range( len(Thetta))])
thp = (np.array(thetp))
thd = (np.array(thetd))
# jobid = os.getenv('SLURM_ARRAY_TASK_ID')
jobid = 0 #int(jobid)-1
######print (str(os.environ["thP"]) +' va '+str(os.environ["thD"]
thP =  thetp[jobid]#int(os.environ["thP"])
thD =  thetd[jobid]# int(os.environ["thD"]) # arr2[int(jobid)]
syspath = os.getcwd()

realization = 5
numberOfBursts = 11

######3ppath = syspath + '/'#'/home/shirin/Desktop/pych/venv/GeneralParameterSearch/'
ppath ='/Users/shirin/Documents/Paper_revision/simulation/Fig3/LowFrequency_random/'
parent_dir = os.path.join(ppath, "threshold"+str(thP)+'_'+str(thD))

os.mkdir(parent_dir)
path = ppath +"threshold"+str(thP)+'_'+str(thD)+'/'
####################Spatio-Temporal discretization########################## 


N_dend = 320
N_spine = 16
NOS_out = 6
NOS_in = N_spine - NOS_out # number of all spines inside the cluster with a L length
NOunstim = 3  # number of unstimulated spines in a L length
L =  80 *1e-6 #10 um
dx = L / (N_dend )
dt = 25 * 1e-5#s   #timestep
T_ms = 130000 #ms
dt_ms = 0.25 # ms
time = int(T_ms/dt_ms)

savestep = 1#time//100






gammaP = 2.3#321.808
gammaD = 1.0#200
Z=2
Deff =  220 * 1e-12  #m^2/s
tau = 0.08#10 * 1e-3 #s
tauspine = 0.08#10 * 1e-3

Faraday = 96485.309 #C/mol
y = 0.11
radius= 1 * 1e-6  #um #den radius
Lneck = 0.5* 1e-6 #um   #0.16 -1.8 mean .98
neckradius= 0.1 * 1e-6 #um 0.06-0.29 /2    0.175 mean /2 =0.0875
spineradius = 0.34 * 1e-6 #  #um 0.26-1.10 /2  mean 0.68 /2 = 0.34


S = np.pi*radius**2
S_prime = np.pi*neckradius**2
Vcomp = dx*np.pi*radius**2
ks =  (4*Deff*neckradius)/Vcomp
Vs = (4/3)*np.pi*spineradius**3
tau_prime = Vs/(4*Deff*neckradius) +(Lneck**2)/(2*Deff)
ks_prime = 1/tau_prime

######backward Euler
alpha = dt*ks
beta = dt*ks_prime
F = Deff * dt / dx ** 2
gamma = dt*y/(Z*Vs*Faraday)
dec = dt/tau
decspine = dt/tauspine
class configuration():
    def __init__(self):#, N_dend,N_spine,NOS_in,NOunstim,L,Deff, dt):

        self.N_dend = N_dend
        self.N_spine = N_spine
        self.L =  L
        self.dx = dx
        self.NOS_in = NOS_in
        self.NOunstim = NOunstim
        self.dt = dt
        self.Deff = Deff
        #backward Euler
        self.alpha = alpha
        self.beta = beta
        self.F = F
        self.gamma = gamma
        self.dec = dec
        self.decspine = decspine
        self.arrangement()
        self.Mat_A()
    def arrangement(self):# N_spine NOS_in
        rnd.seed(seednum)
        
        x_start = 2 #um
        x_end = 9 #um
        extend = 2 #left and right length
        dx_um= self.dx*1e6 #um
        ####clustered spines
        x = list(np.arange(x_start,x_end+dx_um,dx_um) ) # possible locations
        points =list( np.sort(rnd.sample(x, self.NOS_in))) #all spines in a L length
        unstim = rnd.sample(points[1:-1], self.NOunstim)  #unstimulated spines in a L length
        S_stim = list(set(points)- set(unstim)) #  stimulated spines in a L length
        #####outside of the cluster
        x0= list(np.arange(0,x_start,dx_um) )
        NOUT = int(NOS_out/2) # number of unstimulated spines outside the cluster
        x0unstim=list( np.sort(rnd.sample(x0, NOUT)))
        x1= list(np.arange(x_end+dx_um,x_end+extend,dx_um) )
        x1unstim=list( np.sort(rnd.sample(x1, NOUT)))
        S_unstim = list(np.sort(unstim +x0unstim+ x1unstim))

        S_stim_indx = [int(x_loc / dx_um)+140 for x_loc in S_stim]
        S_unstim_indx =[int(x_loc / dx_um)+140 for x_loc in S_unstim]

        spine_list = list(np.sort(S_stim_indx+S_unstim_indx))
        self.spine_list = spine_list
        self.S_stim_indx = S_stim_indx
        self. S_unstim_indx = S_unstim_indx
        return self.spine_list, self.S_stim_indx,self.S_unstim_indx
    

    def Mat_A(self):
        

        adj = np.zeros(self.N_dend)
        adj[self.spine_list] = 1

        sincm = 1
        if self.N_dend == 1:
            sincm = 0
        A = np.zeros((self.N_dend+self.N_spine,self.N_dend+self.N_spine))
        for i in range(self.N_spine+1,self.N_dend+self.N_spine-1):
            A[i, i] = 1 + 2*self.F  +(adj[i-self.N_spine])*self.alpha + self.dec
        A[self.N_spine, self.N_spine] = 1 + (sincm*self.F) + (adj[0])*self.alpha   #first dnd
        A[self.N_dend+self.N_spine-1,self.N_dend+self.N_spine-1]= 1 + (sincm*self.F) + (adj[self.N_dend-1])*self.alpha
        for i in range(self.N_spine,self.N_dend+self.N_spine-1):
            A[i, i + 1] = -self.F
        for i in range(self.N_spine+1,self.N_dend+self.N_spine):
            A[i, i - 1] = -self.F
        for i in range(self.N_spine):
            A[i,i] = 1+self.beta +self.decspine
        for i in range(self.N_spine):
            A[i,self.N_spine+self.spine_list[i]] = - self.alpha
        for i in range(self.N_spine):
            A[self.N_spine + self.spine_list[i],i] = - self.beta
        A = scipy.sparse.csr_matrix(A)
        self.A = scipy.sparse.csr_matrix(A)
        return A
    
    




def Ms_to_step(milsec,dt):
    return int(milsec/(dt * 1e3))

###CURRENT

numberofspike = 1
interval = Ms_to_step(10000,dt) #
input_time = Ms_to_step(1,dt)  # current input tim
taudec = 1*1e-3 #s
I0 = 0.1*1e-12
dts = Ms_to_step(0,dt)  #ACHTUNG!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!must be   Ms_to_step(0.1)








def theta(ar):
    listt=np.zeros(len(ar))
    for values in range(len(ar)):
        if ar[values]>0:
            listt[values]=1
        if ar[values]<=0:
            listt[values]=0
    return listt



for real in range(realization):
    seednum = rnd.randint(1,1000)
    t0_class = tm.time()
    cnfg = configuration()
    A = cnfg.Mat_A()
    arrangement = cnfg.arrangement()
    t1_class = tm.time()
    spine_list = arrangement[0]
    S_stim_indx = arrangement[1]
    S_unstim_indx =  arrangement[2]
    
    stim_spine_indice = [np.where(np.array(spine_list) == i)[0][0] for i in S_stim_indx]
    unstim_spine_indice = [np.where(np.array(spine_list) == i)[0][0] for i in S_unstim_indx]
    weight= np.zeros((N_spine,time))
    w0_range=np.arange(0.1,0.89,0.05)
    w0=[round(rnd.choice(w0_range),2) for i in range(N_spine)]
    weight[:, 0] = w0

    B = []
    for it in range(time):
        B.append(I0 * np.exp(-it * dt / taudec))
    train = np.zeros((N_spine,time))
    # for d in range(numberOfBursts):
    for i in range(numberofspike):
        tt = i * dts
        # train[stim_spine_indice,d * interval + input_time + i * dts] = 1   #stim_spine_indice
        for istim in range(len(stim_spine_indice)):
            nums = random.sample(range(0, 120000), 11)  # sample gives unique numbers

            train[stim_spine_indice[istim],nums] = 1 
    #Noise, instead of coactive activities
    # def shift_array_forward(arr, k):
    #     return [0]*k + arr[:-k]
    # random_delays =[20,30,10,70,10,40,80] #since 7 spines are stimulated
    # for istim in range(len(stim_spine_indice)):
    #     # train[stim_spine_indice[istim],:]=
    #     # print(train[stim_spine_indice[istim]])
    #     train[stim_spine_indice[istim],:]=shift_array_forward(list(np.copy(train[stim_spine_indice[istim],:])), random_delays[istim])

    xs = np.zeros((N_spine, time))
    for spn in range(N_spine):
        cc = np.convolve(train[spn,:], B)
        xs[spn, :] = cc[0:int((len(cc) + 1) / 2)]
    

    parent_dirc = os.path.join(path, 'r_'+str(real))
    os.mkdir(parent_dirc)
    pathsave = parent_dirc+'/'


    b=np.zeros((N_spine+N_dend,time))
    U = np.zeros((N_spine+N_dend,time))
    U[0:N_spine,0] =0
    U[N_spine:N_spine+N_dend,0] = 0


    t0_sim = tm.time()
    for t in range(time-1):
        b[0:N_spine,t] = gamma * xs[:, t] * weight[:,t]
        C = U[:, t] + b[:, t]

        X = scipy.sparse.linalg.spsolve(A,C)
        weight[:,t+1] = weight[:,t] + dt * (gammaP*(1-weight[:,t])*theta(1e6*U[0:N_spine, t]-thP)-gammaD*weight[:,t]*theta(1e6*U[0:N_spine, t]-thD))
        U[:,t+1] = np.asarray(X).flatten()


    t1_sim = tm.time()

    steps=time
    xtime=np.linspace(0,(steps*dt*1e3),steps)

    t0_w = tm.time()
    dfw = pd.DataFrame({'time': xtime })
    for i in range(N_spine):                      # Append columns within for loop
        dfw['spine'+str(i)] = weight[i]
    dfw.to_pickle(pathsave+"w.pkl")
    t1_w = tm.time()

    t0_ca = tm.time()
    dfca = pd.DataFrame({'time': np.arange(0,len(U[0]),savestep) })
    for i in range(N_spine):                      # Append columns within for loop
        dfca['spine'+str(i)] = 1e6*U[i,::savestep]
    dfca.to_pickle(pathsave+"ca.pkl")
    t1_ca = tm.time()

    t0_time_above = tm.time()
    upthetaP = [len((np.where(1e6 * U[0:N_spine, :] > thP))[0]) * dt * 1e3 for upPspine in range(N_spine)]
    upthetaD = [len((np.where(1e6 * U[0:N_spine, :] > thD))[0]) * dt * 1e3 for upDspine in range(N_spine)]

    df = pd.DataFrame({'spine': np.arange(N_spine),'pot': upthetaP,'dep': upthetaD })
    df.to_pickle(pathsave+"time_above_threshold.pkl")
    t1_time_above = tm.time()

    t0_spine_arrangement= tm.time()
    dfspine = pd.DataFrame({'spine_id':spine_list})
    dfstimulated_spine = np.zeros(N_spine)
    dfstimulated_spine[stim_spine_indice]=1
    dfspine['stim'] = dfstimulated_spine
    dfspine.to_pickle(pathsave+"spine_arrangement.pkl")
    t1_spine_arrangement= tm.time()





    t0_I_current = tm.time()
    dfI = pd.DataFrame({'time': np.arange(0,len(xs[0]),savestep) })
    for i in range(N_spine):                      # Append columns within for loop
        dfI['spine'+str(i)] = 1e12*xs[i,::savestep]
    dfI.to_pickle(pathsave+"I_current.pkl")
    t1_I_current = tm.time()
            






